In [1]:
pip install textblob

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import library
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re
from sklearn.preprocessing import OneHotEncoder
from nltk.util import ngrams
from sklearn.pipeline import Pipeline

import pandas as pd
import simplejson as json
from pprint import pprint
from tqdm import tqdm

c:\Users\bobmst\.conda\envs\ds310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\bobmst\.conda\envs\ds310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\bobmst\.conda\envs\ds310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load data

read track infomation

In [3]:
df_track_info = pd.read_pickle("../data/track_info_all.pkl")
print(df_track_info.columns)
df_track_info.head()

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'uri', 'duration_ms'],
      dtype='object')


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226864
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,spotify:track:1AWQoqb9bSvzTjaLralEkT,267267
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600


Read all tracks from all playlists

In [4]:
files = ['mpd.slice.0-999.json'
,'mpd.slice.1000-1999.json'
,'mpd.slice.2000-2999.json'
,'mpd.slice.3000-3999.json'
,'mpd.slice.4000-4999.json'
,'mpd.slice.5000-5999.json'
,'mpd.slice.6000-6999.json'
,'mpd.slice.7000-7999.json'
,'mpd.slice.8000-8999.json'
,'mpd.slice.9000-9999.json']

In [5]:
def get_all_tracks_list(filename):
    with open(filename,'r') as f:
        data = json.load(f)

    ls_df_tracks = []
    for playlist in data['playlists']:
        df_temp = pd.DataFrame.from_records(playlist['tracks'])
        df_temp['pid'] = playlist['pid']
        df_temp['playlist_name'] = playlist['name']
        df_temp['playlist_collaborative'] = playlist['collaborative']
        df_temp['playlist_num_tracks'] = playlist['num_tracks']
        df_temp['playlist_num_followers'] = playlist['num_followers']
        df_temp['playlist_num_artists'] = playlist['num_artists']
        df_temp['playlist_duration_ms'] = playlist['duration_ms']
        ls_df_tracks.append(df_temp)

    return ls_df_tracks

In [6]:
ls_df_tracks = []
for f_name in tqdm(files):
    ls_df_tracks += get_all_tracks_list("../data/"+f_name)
df_tracks = pd.concat(ls_df_tracks, ignore_index=True)
print(df_tracks.columns)
df_tracks

100%|██████████| 10/10 [00:13<00:00,  1.34s/it]


Index(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name',
       'album_uri', 'duration_ms', 'album_name', 'pid', 'playlist_name',
       'playlist_collaborative', 'playlist_num_tracks',
       'playlist_num_followers', 'playlist_num_artists',
       'playlist_duration_ms'],
      dtype='object')


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,pid,playlist_name,playlist_collaborative,playlist_num_tracks,playlist_num_followers,playlist_num_artists,playlist_duration_ms
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0,Throwbacks,false,52,1,37,11532414
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0,Throwbacks,false,52,1,37,11532414
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0,Throwbacks,false,52,1,37,11532414
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0,Throwbacks,false,52,1,37,11532414
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0,Throwbacks,false,52,1,37,11532414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664707,50,Keith Urban,spotify:track:6ZOPiKQeibCn7fP8dncucL,spotify:artist:0u2FHSq3ln94y5Q57xazwf,Blue Ain't Your Color,spotify:album:1r7ABqzNXQnUPAH3ZjrHMn,230600,Ripcord,9999,Country jams,false,55,3,33,11604099
664708,51,Dustin Lynch,spotify:track:7pxhKtuTwofDIdgHx2DcVK,spotify:artist:1dID9zgn0OV0Y8ud7Mh2tS,Seein' Red,spotify:album:23cuZhPWDfX1uKD4qwuv7t,194893,Current Mood,9999,Country jams,false,55,3,33,11604099
664709,52,Sam Hunt,spotify:track:7mldq42yDuxiUNn08nvzHO,spotify:artist:2kucQ9jQwuD8jWdtR9Ef38,Body Like A Back Road,spotify:album:2N7kidh1wA9EoLdf16QWrz,165386,Body Like A Back Road,9999,Country jams,false,55,3,33,11604099
664710,53,Little Big Town,spotify:track:23TxRN09aR1RB0G0tFoT0b,spotify:artist:3CygdxquGHurS7f9LjNLkv,Better Man,spotify:album:2aQOzEjLzPkffXDwREXdAh,263120,The Breaker,9999,Country jams,false,55,3,33,11604099


In [7]:
# select features
df_tracks[['artist_name', 'track_uri', 'artist_uri', 'track_name',
       'album_uri', 'album_name']].drop_duplicates()

,artist_name,track_uri,artist_uri,track_name,album_uri,album_name
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,The Cookbook
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,In The Zone
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,Dangerously In Love (Alben für die Ewigkeit)
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justified
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Hot Shot
...,...,...,...,...,...,...
664635,NateWantsToBattle,spotify:track:4cKPzfOUIZWXhu3GDnESjL,spotify:artist:0Vb15td3iKkAzdGD5Sj9Ky,Reluctant Heroes,spotify:album:05XnUTo4HrVy5eLzqcD0lg,Reluctant Heroes
664638,Dima Lancaster,spotify:track:0VICzaRdctD8XGfDstBjQY,spotify:artist:7tSqWbPIUCl6bTbPoxLXGn,Your Lie in April - Medley,spotify:album:4CKBpgzwIL7NPJrOvltCHj,Your Lie in April - Medley
664640,Set It Off,spotify:track:0B4YuevrDNhh71edZrFiOs,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Uncontainable,spotify:album:0jGFC44cRkjE2b21aMEbIo,Upside Down
664641,Set It Off,spotify:track:7i9q0VPbENCoYnJUsJhX9d,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Duality,spotify:album:5AKycvv2TUUh9h0KIletJt,Duality


In [8]:
df_track_info

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms
0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226864
1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800
2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933
3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,spotify:track:1AWQoqb9bSvzTjaLralEkT,267267
4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170084,0.551,0.822,11,-7.018,0,0.0614,0.001540,0.000000,0.2150,0.360,127.043,spotify:track:4cKPzfOUIZWXhu3GDnESjL,265039
170085,0.480,0.761,6,-4.628,1,0.0367,0.120000,0.000000,0.2960,0.359,160.020,spotify:track:0VICzaRdctD8XGfDstBjQY,368767
170086,0.402,0.767,11,-6.718,0,0.1710,0.000077,0.000003,0.3930,0.558,178.293,spotify:track:0B4YuevrDNhh71edZrFiOs,179267
170087,0.678,0.826,0,-4.742,0,0.0591,0.005240,0.000000,0.0951,0.706,139.989,spotify:track:7i9q0VPbENCoYnJUsJhX9d,242347


In [9]:
df_track_with_info = df_track_info.merge(df_tracks[['artist_name', 'track_uri', 'artist_uri', 'track_name',
       'album_uri', 'album_name']].drop_duplicates(),left_on='uri',right_on='track_uri',how='left')
df_track_with_info

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,artist_name,track_uri,artist_uri,track_name,album_uri,album_name
0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226864,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,The Cookbook
1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,In The Zone
2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,Dangerously In Love (Alben für die Ewigkeit)
3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,spotify:track:1AWQoqb9bSvzTjaLralEkT,267267,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justified
4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Hot Shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170084,0.551,0.822,11,-7.018,0,0.0614,0.001540,0.000000,0.2150,0.360,127.043,spotify:track:4cKPzfOUIZWXhu3GDnESjL,265039,NateWantsToBattle,spotify:track:4cKPzfOUIZWXhu3GDnESjL,spotify:artist:0Vb15td3iKkAzdGD5Sj9Ky,Reluctant Heroes,spotify:album:05XnUTo4HrVy5eLzqcD0lg,Reluctant Heroes
170085,0.480,0.761,6,-4.628,1,0.0367,0.120000,0.000000,0.2960,0.359,160.020,spotify:track:0VICzaRdctD8XGfDstBjQY,368767,Dima Lancaster,spotify:track:0VICzaRdctD8XGfDstBjQY,spotify:artist:7tSqWbPIUCl6bTbPoxLXGn,Your Lie in April - Medley,spotify:album:4CKBpgzwIL7NPJrOvltCHj,Your Lie in April - Medley
170086,0.402,0.767,11,-6.718,0,0.1710,0.000077,0.000003,0.3930,0.558,178.293,spotify:track:0B4YuevrDNhh71edZrFiOs,179267,Set It Off,spotify:track:0B4YuevrDNhh71edZrFiOs,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Uncontainable,spotify:album:0jGFC44cRkjE2b21aMEbIo,Upside Down
170087,0.678,0.826,0,-4.742,0,0.0591,0.005240,0.000000,0.0951,0.706,139.989,spotify:track:7i9q0VPbENCoYnJUsJhX9d,242347,Set It Off,spotify:track:7i9q0VPbENCoYnJUsJhX9d,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Duality,spotify:album:5AKycvv2TUUh9h0KIletJt,Duality


In [12]:
df_track_with_info.set_index('uri').to_pickle('../data/track_info_with_artist_album.pkl')

## Feature Extraction

Sentimental analysis on track, album name

In [11]:
def getSubjectivity(text):
  '''
  Getting the Subjectivity using TextBlob
  '''
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  '''
  Getting the Polarity using TextBlob
  '''
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  '''
  Categorizing the Polarity & Subjectivity score
  '''
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'
      
def sentiment_analysis(df, text_col):
  '''
  Perform sentiment analysis on text
  ---
  Input:
  df (pandas dataframe): Dataframe of interest
  text_col (str): column of interest
  '''
  df[text_col+'_subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df[text_col+'_polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [12]:
df_track_with_info = sentiment_analysis(df_track_with_info, "track_name")
df_track_with_info = sentiment_analysis(df_track_with_info, "album_name")
df_track_with_info 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,artist_name,track_uri,artist_uri,track_name,album_uri,album_name,track_name_subjectivity,track_name_polarity,album_name_subjectivity,album_name_polarity
0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,...,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,The Cookbook,low,Neutral,low,Neutral
1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,...,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,In The Zone,low,Neutral,low,Neutral
2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,...,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,Dangerously In Love (Alben für die Ewigkeit),high,Negative,high,Positive
3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,...,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justified,low,Neutral,high,Positive
4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,...,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Hot Shot,low,Neutral,high,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170084,0.551,0.822,11,-7.018,0,0.0614,0.001540,0.000000,0.2150,0.360,...,NateWantsToBattle,spotify:track:4cKPzfOUIZWXhu3GDnESjL,spotify:artist:0Vb15td3iKkAzdGD5Sj9Ky,Reluctant Heroes,spotify:album:05XnUTo4HrVy5eLzqcD0lg,Reluctant Heroes,low,Neutral,low,Neutral
170085,0.480,0.761,6,-4.628,1,0.0367,0.120000,0.000000,0.2960,0.359,...,Dima Lancaster,spotify:track:0VICzaRdctD8XGfDstBjQY,spotify:artist:7tSqWbPIUCl6bTbPoxLXGn,Your Lie in April - Medley,spotify:album:4CKBpgzwIL7NPJrOvltCHj,Your Lie in April - Medley,low,Neutral,low,Neutral
170086,0.402,0.767,11,-6.718,0,0.1710,0.000077,0.000003,0.3930,0.558,...,Set It Off,spotify:track:0B4YuevrDNhh71edZrFiOs,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Uncontainable,spotify:album:0jGFC44cRkjE2b21aMEbIo,Upside Down,low,Neutral,low,Negative
170087,0.678,0.826,0,-4.742,0,0.0591,0.005240,0.000000,0.0951,0.706,...,Set It Off,spotify:track:7i9q0VPbENCoYnJUsJhX9d,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,Duality,spotify:album:5AKycvv2TUUh9h0KIletJt,Duality,low,Neutral,low,Neutral


tf-idf

In [13]:
df_track_with_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170089 entries, 0 to 170088
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   danceability             170089 non-null  float64
 1   energy                   170089 non-null  float64
 2   key                      170089 non-null  int64  
 3   loudness                 170089 non-null  float64
 4   mode                     170089 non-null  int64  
 5   speechiness              170089 non-null  float64
 6   acousticness             170089 non-null  float64
 7   instrumentalness         170089 non-null  float64
 8   liveness                 170089 non-null  float64
 9   valence                  170089 non-null  float64
 10  tempo                    170089 non-null  float64
 11  uri                      170089 non-null  object 
 12  duration_ms              170089 non-null  int64  
 13  artist_name              170089 non-null  object 
 14  trac

In [14]:
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(df_track_with_info[['artist_name']].apply(lambda x: " ".join(x).lower()))
artist_df = pd.DataFrame(tfidf_matrix.toarray())
artist_df

,0,1,2,3,4,5,6,7,8,9,...,28897,28898,28899,28900,28901,28902,28903,28904,28905,28906
0,0.001202,0.00006,0.00006,0.00006,0.000541,0.00006,0.002706,0.000601,0.00006,0.00006,...,0.00006,0.00018,0.00006,0.00006,0.00006,0.00006,0.00006,0.00006,0.00012,0.00024


In [15]:
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(df_track_with_info[['album_name']].apply(lambda x: " ".join(x).lower()))
album_df = pd.DataFrame(tfidf_matrix.toarray())
album_df

,0,1,2,3,4,5,6,7,8,9,...,32499,32500,32501,32502,32503,32504,32505,32506,32507,32508
0,0.000178,0.000917,0.000025,0.000025,0.000407,0.000127,0.000051,0.000076,0.000025,0.000127,...,0.000153,0.000025,0.000025,0.000025,0.000025,0.000025,0.000127,0.000025,0.000051,0.000025


In [16]:
df_track_with_info.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'uri', 'duration_ms', 'artist_name', 'track_uri', 'artist_uri',
       'track_name', 'album_uri', 'album_name', 'track_name_subjectivity',
       'track_name_polarity', 'album_name_subjectivity',
       'album_name_polarity'],
      dtype='object')

one-hot encoding

In [17]:
df_track_with_info_2 = df_track_with_info[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms',
        'track_name_subjectivity',
       'track_name_polarity', 'album_name_subjectivity',
       'album_name_polarity']].reset_index()
df_track_with_info_2

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_name_subjectivity,track_name_polarity,album_name_subjectivity,album_name_polarity
0,0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,226864,low,Neutral,low,Neutral
1,1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,198800,low,Neutral,low,Neutral
2,2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,235933,high,Negative,high,Positive
3,3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,267267,low,Neutral,high,Positive
4,4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,227600,low,Neutral,high,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170084,170084,0.551,0.822,11,-7.018,0,0.0614,0.001540,0.000000,0.2150,0.360,127.043,265039,low,Neutral,low,Neutral
170085,170085,0.480,0.761,6,-4.628,1,0.0367,0.120000,0.000000,0.2960,0.359,160.020,368767,low,Neutral,low,Neutral
170086,170086,0.402,0.767,11,-6.718,0,0.1710,0.000077,0.000003,0.3930,0.558,178.293,179267,low,Neutral,low,Negative
170087,170087,0.678,0.826,0,-4.742,0,0.0591,0.005240,0.000000,0.0951,0.706,139.989,242347,low,Neutral,low,Neutral


In [19]:
df_track_with_info_2

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_name_subjectivity,track_name_polarity,album_name_subjectivity,album_name_polarity
0,0,0.904,0.813,4,-7.105,0,0.1210,0.031100,0.006970,0.0471,0.810,125.461,226864,low,Neutral,low,Neutral
1,1,0.774,0.838,5,-3.914,0,0.1140,0.024900,0.025000,0.2420,0.924,143.040,198800,low,Neutral,low,Neutral
2,2,0.664,0.758,2,-6.583,0,0.2100,0.002380,0.000000,0.0598,0.701,99.259,235933,high,Negative,high,Positive
3,3,0.892,0.714,4,-6.055,0,0.1410,0.201000,0.000234,0.0521,0.817,100.972,267267,low,Neutral,high,Positive
4,4,0.853,0.606,0,-4.596,1,0.0713,0.056100,0.000000,0.3130,0.654,94.759,227600,low,Neutral,high,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170084,170084,0.551,0.822,11,-7.018,0,0.0614,0.001540,0.000000,0.2150,0.360,127.043,265039,low,Neutral,low,Neutral
170085,170085,0.480,0.761,6,-4.628,1,0.0367,0.120000,0.000000,0.2960,0.359,160.020,368767,low,Neutral,low,Neutral
170086,170086,0.402,0.767,11,-6.718,0,0.1710,0.000077,0.000003,0.3930,0.558,178.293,179267,low,Neutral,low,Negative
170087,170087,0.678,0.826,0,-4.742,0,0.0591,0.005240,0.000000,0.0951,0.706,139.989,242347,low,Neutral,low,Neutral


In [22]:
attr_countineous = ['index', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms']
attr_categorical = ['track_name_subjectivity',
       'track_name_polarity', 'album_name_subjectivity',
       'album_name_polarity']
df_track_with_info_2.columns


Index(['index', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'track_name_subjectivity',
       'track_name_polarity', 'album_name_subjectivity',
       'album_name_polarity'],
      dtype='object')

In [34]:
num_pipeline = Pipeline(
    [
        ("average_imputer", SimpleImputer(strategy="mean")),
        ("mm_scaler", MinMaxScaler()),
    ]
)

cate_pipeline = Pipeline(
    [
        ("string_imputer", SimpleImputer(strategy="constant", fill_value="")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)
full_pipeline = ColumnTransformer(
    [
        ("num", num_pipeline, attr_countineous),
        ("cat", cate_pipeline, attr_categorical),
    ]
)

In [37]:
df_track_with_info_3 = pd.DataFrame(full_pipeline.fit_transform(df_track_with_info_2))
df_track_with_info_3

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.000000,0.912210,0.813,0.363636,0.842733,0.0,0.125649,0.031225,0.006977,0.0471,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.000006,0.781029,0.838,0.454545,0.893573,0.0,0.118380,0.025000,0.025025,0.2420,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.000012,0.670030,0.758,0.181818,0.851050,0.0,0.218069,0.002390,0.000000,0.0598,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.000018,0.900101,0.714,0.363636,0.859462,0.0,0.146417,0.201807,0.000234,0.0521,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.000024,0.860747,0.606,0.000000,0.882707,1.0,0.074039,0.056325,0.000000,0.3130,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170084,0.999976,0.556004,0.822,1.000000,0.844119,0.0,0.063759,0.001546,0.000000,0.2150,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
170085,0.999982,0.484359,0.761,0.545455,0.882197,1.0,0.038110,0.120482,0.000000,0.2960,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
170086,0.999988,0.405651,0.767,1.000000,0.848899,0.0,0.177570,0.000078,0.000003,0.3930,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
170087,0.999994,0.684157,0.826,0.000000,0.880381,0.0,0.061371,0.005261,0.000000,0.0951,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [41]:
df_track_with_info_3 = df_track_with_info_3.set_index(df_track_with_info['uri'])

In [42]:
df_track_with_info_3.to_pickle('../data/complete_feature.pkl')